# 2. Using CodeCarbon -- RLS

### Authors: *Pedro Andrade, Ivanovitch Silva*

In [1]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 KB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 3.8 MB/s eta 0:00:00


In [2]:
!pip install imutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install codecarbon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.9/174.9 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 3.6 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd drive/MyDrive/UFRN/Doutorado/CodeCarbon/fuel_data

/content/drive/MyDrive/UFRN/Doutorado/CodeCarbon/fuel_data


In [6]:
ls

'1.[TEDAEnsemble] WandB with Window.ipynb'        Results_ACM.ipynb
'2.[RLS] WandB with Window.ipynb'                 Results.ipynb
'3.[XGBoosting] Wandb with Window.ipynb'          table_cnn_acm.csv
'4.[Linear Regression] Wandb with Window.ipynb'   table_lreg.csv
'5.[LSTM] Wandb with Window.ipynb'                table_lstm.csv
'6.[CNN] Wandb with Window.ipynb'                 table_rls_acm.csv
 acm_cnn_table.csv                                table_rls.csv
 acm_rls_table.csv                                table_teda.csv
 acm_teda_table.csv                               table_teda_regressor_acm.csv
 dados_household.xlsx                             table_xgb.csv
 emissions.csv                                    TedaEnsemble.py
 full_data.xlsx                                   treating.py
 padasip/                                         wandb/
 __pycache__/


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
import statsmodels.api as sm
import padasip as pa

from sklearn.metrics import mean_squared_error

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import datetime

import wandb
import logging

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import KFold, StratifiedKFold,TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from IPython.display import Image
from sklearn.linear_model import LinearRegression
from sklearn import tree
from xgboost import XGBClassifier, XGBRegressor
import xgboost as xgb
%config IPCompleter.greedy=True

In [9]:
from codecarbon import EmissionsTracker

## Implemented modules and Padasip module

In [10]:
from padasip.filters.base_filter import AdaptiveFilter
from TedaEnsemble import DataCloud, AutoCloud

## Data loading:

In [11]:
!wandb login 40f0acf51f839c9bfbed5c1f24e945ed8245e110

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [12]:
#dv = pd.read_csv("temperatures.csv")
#dv = pd.read_excel("full_data.xlsx")
df = pd.read_excel("full_data.xlsx")

In [13]:
# Initialize a new W&B run to track this job
run = wandb.init(project="TEDARegressor", job_type="dataset-creation")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pedrohmeira (embarca-ai). Use `wandb login --relogin` to force relogin


In [14]:
# Create a sample dataset to log as an artifact
#f = open('my-dataset.txt', 'w')
#f.write('Imagine this is a big dataset.')
#f.close()

# Create a new artifact, which is a sample dataset
dataset = wandb.Artifact('full_data', type='dataset')
# Add files to the artifact, in this case a simple text file
dataset.add_file('full_data.xlsx')
# Log the artifact to save it as an output of this run
run.log_artifact(dataset)

In [15]:
# configure logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%d-%m-%Y %H:%M:%S')

# reference for a logging obj
logger = logging.getLogger()

## Our data isn't treated. So, we have one step before:

In [16]:
#df['fuel_mL'] = 1000*df['fuel diff']

In [17]:
df.head()

,Trip Time,Latitude,Longitude,RPM,Fuel level,fuel,Trip Distance(OBD),Speed,Temperature,Pressure,fuel diff,Trip Distance,Distance Diff,Dist Diff Calc,Trip Distance Calc (km)
0,0,-5.776461,-35.204017,881.5,47.843136,0.014118,0.000087,0,317.15,37.000015,0.000000,0.000087,0.000000,0.000000,0.000000
1,1,-5.776427,-35.204121,844.5,47.843136,0.014377,0.000234,1,317.15,48.000020,0.000260,0.000234,0.000147,0.012103,0.012103
2,2,-5.776427,-35.204121,836.0,47.843136,0.014730,0.000920,4,317.15,54.000022,0.000353,0.000920,0.000686,0.000000,0.012103
3,3,-5.776358,-35.204040,1042.5,47.843136,0.015100,0.002066,5,317.15,53.000022,0.000369,0.002066,0.001146,0.011725,0.023828
4,4,-5.776358,-35.204040,1519.5,47.843136,0.015669,0.004019,8,317.15,47.000020,0.000569,0.004019,0.001953,0.000000,0.023828


In [18]:
def treating_dataset(name_series, window, N_splits):
    sum_series = name_series
    window = window
    model_data = pd.DataFrame()
    model_data['t'] = [x for x in sum_series]
    for i in range(1, window+1):
        model_data['t + ' + str(i)] = model_data['t'].shift(-i)

    model_data.dropna(axis=0, inplace=True)
    
    X = model_data.iloc[:,0:-1].values
    y = model_data.iloc[:, -1].values
    
    tscv = TimeSeriesSplit(n_splits=N_splits)
    
    for train_index, test_index in tscv.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
    
    return X_train, y_train, X_test, y_test

In [19]:
metric_dict = {
    'name': 'mse_RLS',
    'goal': 'minimize'   
    }

In [20]:
parameters_dict = {
    #RLS
    'Window': {
        'values': [2,3,4,5,6]
        },
    'Factor': {
        'values': [0.000001, 0.1, 0.25, 0.5, 0.75, 0.9, 1]
        },
}

In [21]:
seed = 42
sweep_config = {
    "method": "grid",
    "metric": metric_dict,
    "parameters": parameters_dict,

}
sweep_id = wandb.sweep(sweep_config, project="TEDARegressor")

Create sweep with ID: z1g462gd
Sweep URL: https://wandb.ai/embarca-ai/TEDARegressor/sweeps/z1g462gd


In [22]:
sum_series=df['fuel diff']

#RLS
MSE_RLS = []

In [23]:
def train():
    with wandb.init() as run:
        
        # create codecarbon tracker
        # codecarbon is too much verbose, change the log level for more info
        tracker = EmissionsTracker(log_level="critical")
        tracker.start()

        X_train, y_train, X_test, y_test = treating_dataset(name_series = sum_series, window=wandb.config.Window, N_splits = 5)
        
        #RLS
        fRLS = pa.filters.FilterRLS(wandb.config.Window, mu=wandb.config.Factor, w="zeros")
        Y_RLS = []
        
        

        #X = X.T
        t_ant = X_test[0]
        count=0

        for t in X_test:

            #RLS
            y = fRLS.predict(t)
            fRLS.adapt(t[-1], t_ant) 
            t_ant = t
            Y_RLS.append(y)
            count+=1

        # get co2 emissions from tracker
        # "CO2 emission (in Kg)"
        emissions = tracker.stop() #CONFERIR SE É AQUI MESMO
      
        #RLS
        mse_RLS = mean_squared_error(y_test[1:-1], Y_RLS[2:])
        
        MSE_RLS.append(mse_RLS)
        
        run.summary['mse_RLS'] = mse_RLS
        wandb.log({"mse_RLS": mse_RLS})

        # energy unit is kWh
        run.summary["Energy_Consumed"] = tracker.final_emissions_data.energy_consumed
        run.summary["Energy_RAM"] = tracker.final_emissions_data.ram_energy
        run.summary["Energy_GPU"] = tracker.final_emissions_data.gpu_energy
        run.summary["Energy_CPU"] = tracker.final_emissions_data.cpu_energy
        # kg
        run.summary["CO2_Emissions"] = tracker.final_emissions_data.emissions

In [24]:
wandb.agent(sweep_id, train)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: q9dlpacc with config:
wandb: 	Factor: 1e-06
wandb: 	Window: 2


mse_RLS,▁
CO2_Emissions,0.0
Energy_CPU,0.0
Energy_Consumed,0.0
Energy_GPU,0
Energy_RAM,0.0
mse_RLS,0.0


wandb: Agent Starting Run: pt4ljdzz with config:
wandb: 	Factor: 1e-06
wandb: 	Window: 3


mse_RLS,▁
CO2_Emissions,0.0
Energy_CPU,0.0
Energy_Consumed,0.0
Energy_GPU,0
Energy_RAM,0.0
mse_RLS,0.0


wandb: Agent Starting Run: 6ij0566c with config:
wandb: 	Factor: 1e-06
wandb: 	Window: 4


...

wandb: Agent Starting Run: g7mlfebr with config:
wandb: 	Factor: 1
wandb: 	Window: 6


mse_RLS,▁
CO2_Emissions,0.0
Energy_CPU,0.0
Energy_Consumed,0.0
Energy_GPU,0
Energy_RAM,0.0
mse_RLS,0.0


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


Error in callback <function _WandbInit._pause_backend at 0x7f144aa3c160> (for post_run_cell):


BrokenPipeError: ignored

In [ ]:
run.finish()

In [ ]:
min(MSE_RLS)

In [ ]:
max(MSE_RLS)

In [ ]:
np.array(MSE_RLS).mean()

In [ ]:
np.array(MSE_RLS).std()